## What is clickbait?
Click bait is a form of false advertisement which uses a hyperlink text or a thumbnail link that is designed to attract attention and entice users to follow that link and read, view or listen to the linked piece of online content, with a defining charecteristic of being deceptive, typically sensationalized or misleading.
https://towardsdatascience.com/text-classification-with-extremely-small-datasets-333d322caee2

In [1]:
import numpy as np
import pandas as pd

In [6]:
ls

 Volume in drive C has no label.
 Volume Serial Number is B4CF-A33C

 Directory of c:\Users\Arnab\Documents\GitHub\arnabpro007.github.io\Data Analysis\Text Classification with small dataset

31-08-2021  19:58    <DIR>          .
31-08-2021  19:58    <DIR>          ..
15-12-2016  17:25           923,965 clickbait_data
15-12-2016  17:25           856,139 non_clickbait_data
31-08-2021  16:00               579 Text Classification with small dataset.ipynb
               3 File(s)      1,780,683 bytes
               2 Dir(s)  950,237,097,984 bytes free


In [9]:
#First challenge I am facing is even creating this dataset, as I can't use pandas to directly read it.
with open("clickbait_data") as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]

#we are falsely adding white space in the list as well so we will remove those
click_bait = []
for ele in lines:
    if ele.strip():
        click_bait.append(ele)

In [11]:
len(click_bait)

15999

In [17]:
file.close()

In [19]:
with open("non_clickbait_data",encoding="utf8") as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]

#we are falsely adding white space in the list as well so we will remove those
non_click_bait = []
for ele in lines:
    if ele.strip():
        non_click_bait.append(ele)

In [20]:
len(non_click_bait)

16001

In [21]:
file.close()

In [30]:
temp = non_click_bait + click_bait

In [31]:
len(temp)

32000

In [39]:
words = ["clickbait","not-clickbait"]
y =[]
for i in range(15999):
    y = y + [words[0]]
for i in range(16001):
    y = y + [words[1]]

In [42]:
data = {"title":temp,"label":y}

data = pd.DataFrame(data)

In [43]:
display(data)

,title,label
0,Bill Changing Credit Card Rules Is Sent to Oba...,clickbait
1,"In Hollywood, the Easy-Money Generation Toughe...",clickbait
2,1700 runners still unaccounted for in UK's Lak...,clickbait
3,Yankees Pitchers Trade Fielding Drills for Put...,clickbait
4,Large earthquake rattles Indonesia; Seventh in...,clickbait
...,...,...
31995,"There Was A Mini ""Sisterhood Of The Traveling ...",not-clickbait
31996,21 Dogs Who Are Thankful For Their Best Friends,not-clickbait
31997,People Are Proving No Dick Is Too Big By Dropp...,not-clickbait
31998,"I'm An Atheist, But I'm Not",not-clickbait


In [44]:
#Now we will shuffle it and pick 10050 of it
data = data.reindex(np.random.permutation(data.index))
data = data[:10050]

In [45]:
display(data)

,title,label
9708,Doctors Face Fiscal Squeeze for Treatment,clickbait
26860,7 Perfectly Appropriate Responses To Someone A...,not-clickbait
24266,13 Insanely Easy Three-Ingredient Desserts,not-clickbait
8296,Home-invaders pose as NYC police,clickbait
7848,Emergency spacewalks planned to fix Internatio...,clickbait
...,...,...
27242,"Can You Remember The First Episode Of ""Full Ho...",not-clickbait
17915,Let's See How Well You Remember The First Epis...,not-clickbait
17761,How Well Do You Remember The Lyrics To Panic! ...,not-clickbait
23308,Are You More Severus Snape Or Albus Dumbledore,not-clickbait


In [51]:
#Lets save the data
data.to_csv("Data.csv")

In [2]:
data = pd.read_csv("Data.csv")

In [3]:
#Now lets split the data
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, shuffle = True, stratify = data.label, train_size = 50/data.shape[0], random_state = 50)


In [4]:
train.shape

(50, 3)

In [5]:
test.shape

(10000, 3)

In [6]:
train.sample(10)

,Unnamed: 0,title,label
2848,4309,Facebook to share the technology 'behind its s...,clickbait
8484,6184,Rockets Defy Logic by Dominating Without Yao,clickbait
8754,778,Thai political talk show taken off during medi...,clickbait
9826,21342,We Need To Talk About How Sexy Jared Keeso Is,not-clickbait
7669,21821,17 Heartbreaking Confessions From People Who C...,not-clickbait
3125,3010,Mortar exchange kills thirteen in Somali capital,clickbait
8745,26515,#COCKROACHESANDCHER Is The Most Important Hash...,not-clickbait
1822,3066,Glastonbury headliners announced,clickbait
9869,15111,SLM shareholders led by Martin Garbus sue Spid...,clickbait
3319,18895,This Trailer For Bromances Is Everything You Need,not-clickbait


## Adversarial Validation

In [6]:
from scipy import sparse
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold


def adversarial_validation(X, Y, n_splits = 10):
    
    # Combine both datasets
    sparse_merge = sparse.vstack((X, Y))
    
    # Label the datasets
    y = np.array([0 for _ in range(X.shape[0])] + [1 for _ in range(Y.shape[0])])
    
    # Do 10 Fold CV 
    kfold = StratifiedKFold(n_splits = n_splits, shuffle = True)

    lr_auc = np.array([])
    rf_auc = np.array([])
    for train_idx, test_idx in kfold.split(sparse_merge, y):
    
        #Run Log Reg
        x_train, y_train = sparse_merge[train_idx], y[train_idx]
        x_test, y_test = sparse_merge[test_idx], y[test_idx]
        
        log_reg = SGDClassifier(loss = 'log')
        log_reg.fit(x_train, y_train)
        y_test_prob = log_reg.predict_proba(x_test)[:,1]
        lr_auc = np.append(lr_auc, roc_auc_score(y_test, y_test_prob))
        # Run RF
        rf = RandomForestClassifier(n_estimators = 100, n_jobs = -1)
        rf.fit(x_train, y_train)
        y_test_prob = rf.predict_proba(x_test)[:,1]
        rf_auc = np.append(rf_auc, roc_auc_score(y_test, y_test_prob))

    
    # Display results
    print('Logisitic Regression AUC : {:.3f}'.format(lr_auc.mean()))
    print('Random Forest AUC : {:.3f}'.format(rf_auc.mean())) 

In [7]:
# We will use bag of words to encode the title before using the validation

from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()

x_train = bow.fit_transform(train.title.values)
x_test = bow.transform(test.title.values)
adversarial_validation(x_train, x_test[:50])

Logisitic Regression AUC : 0.444
Random Forest AUC : 0.526


## Simple Exploratory Data Analysis

In [4]:
print('Train Positive Class % : {:.1f}'.format((sum(train.label == 'clickbait')/train.shape[0])*100))
print('Test Positive Class % : {:.1f}'.format((sum(test.label == 'clickbait')/test.shape[0])*100))
print('Train Size: {}'.format(train.shape[0]))
print('Test Size: {}'.format(test.shape[0]))

Train Positive Class % : 50.0
Test Positive Class % : 50.0
Train Size: 50
Test Size: 10000
